In [1]:
# load the import libraries
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import os
import string
import datetime
from transformers import pipeline
from transformers import AutoTokenizer,AutoModelForSequenceClassification
from collections import defaultdict
from tqdm import tqdm
import tensorflow as tf


RuntimeError: Failed to import transformers.pipelines because of the following error (look up to see its traceback):
This version of TensorFlow Probability requires TensorFlow version >= 2.15; Detected an installation of version 2.10.0. Please upgrade TensorFlow to proceed.

In [ ]:
news_file_gp_list = ['../Scraper/News-Money-control/final-data/gapped_data\\short_0.csv',
 '../Scraper/News-Money-control/final-data/gapped_data\\short_1.csv',
 '../Scraper/News-Money-control/final-data/gapped_data\\short_2.csv',
 '../Scraper/News-Money-control/final-data/gapped_data\\short_3.csv',
 '../Scraper/News-Money-control/final-data/gapped_data\\short_4.csv']

temp_comb_g_n_df = []

for gn_f in news_file_gp_list:
    df = pd.read_csv(gn_f)
    temp_comb_g_n_df.append(df)
    
comb_g_df = pd.concat(temp_comb_g_n_df)
comb_g_df.drop('published_at.1',axis=1,inplace=True)
comb_g_df.set_index('published_at',inplace=True)
comb_g_df.dropna(subset=['description'],inplace=True)

In [2]:
pipe = pipeline('text-classification',model='ProsusAI/finbert')
tokenizer = AutoTokenizer.from_pretrained('ProsusAI/finbert')

model = AutoModelForSequenceClassification.from_pretrained('ProsusAI/Finbert') 

C:\Users\jeetc\miniconda3\envs\fooocus\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [16]:
import json

# Define a function to process a batch of data and save sentiments
def process_and_save_batch(batch_df, text_label, save_folder):
    sentiments = defaultdict(lambda: {'pos': [], 'neg': [], 'neu': []})

    for index, row in tqdm(batch_df.iterrows(), total=len(batch_df)):
        text = [row[text_label][:400]]

        inputs = tokenizer(text, return_tensors='pt')
        outputs = model(**inputs)
        probs = outputs.logits.softmax(dim=1)

        pos = probs[:, 0].tolist()
        neg = probs[:, 1].tolist()
        neu = probs[:, 2].tolist()

        sentiments[index]['pos'].extend(pos)
        sentiments[index]['neg'].extend(neg)
        sentiments[index]['neu'].extend(neu)

    # Save sentiments to a file in the specified folder
    save_path = f"sentiments/sentiments_{save_folder}.json".replace(" ","")
    with open(save_path, 'w') as f:
        json.dump(sentiments, f)

In [5]:
batches = list(range(0,1700000,100000))

In [ ]:
for i in range(len(batches)):
    print(batches[i],batches[i+1])
    process_and_save_batch(comb_g_df[batches[i]:batches[i+1]],'description',i)

0 100000


 10%|███████▏                                                               | 10094/100000 [7:57:45<3:44:08,  6.69it/s]